In [ ]:
!pip install langchain_google_vertexai langgraph

# Controlled generation

Controlled generation using `ChatVertexAI` can be set up using the following arguments:

- `response_schema`: Schema in OpenAPI format
- `response_mime_type`: This argument must be set to `"application\json"`

In the following example, we'll instruct the model to structure its response as an array of objects, each containing the properties city_name and population

In [1]:
from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(
    model_name="gemini-1.5-flash-001", 
    response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "city_name": {
                "type": "string",
            },
            "population": {
                "type": "number"
            }
        },
        "required": ["city_name", "population"],
      },
    }, 
    response_mime_type = "application/json",
)

Now upon model invocation, we will receive an `AIMessage`. The property `content` of the message will be a json string with the specified structure:

In [2]:
model.invoke("What are the 3 biggest cities in france")

AIMessage(content='[{"city_name": "Paris", "population": 2140526}, {"city_name": "Marseille", "population": 861549}, {"city_name": "Lyon", "population": 516092}]\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 17, 'candidates_token_count': 60, 'total_token_count': 77, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.10022381146748861, 

Aditionally, we can make use of `langchain_core.ouput_parsers` to create a chain that automatically parses the generated json string into a dictionary.

In [3]:
from langchain_core.output_parsers import SimpleJsonOutputParser

parser = SimpleJsonOutputParser()

chain = model | parser

chain.invoke("What are the 3 biggest cities in france")

[{'city_name': 'Paris', 'population': 2141.96},
 {'city_name': 'Lyon', 'population': 516.09},
 {'city_name': 'Marseille', 'population': 870.72}]

Controlled generation can also be achieved using Pydantic models rather than OpenAPI schemas. Rather than specifying keyword arguments, we can use the method `with_structured_output` and a Pydanctic model. If we reproduce the example above:

In [4]:
from pydantic import BaseModel, Field

class City(BaseModel):
    """ Represents a city with its population and name.
    """
    city_name: str = Field(description="Name of the city")
    population: int = Field(description="Total population of the city")

model = ChatVertexAI(model_name="gemini-1.5-flash-001").with_structured_output(City, method="json_mode")

model.invoke("What are the 3 biggest cities in france")

City(city_name='Paris', population=2140526)

In this case, the output of the model will be the Pydantic object defined.